In [1]:
%matplotlib inline

# plotting
import matplotlib as mpl
mpl.style.use('ggplot')
import matplotlib.pyplot as plt

# math and data manipulation
import numpy as np
import pandas as pd

# to handle paths
from pathlib import Path

# set random seeds 
import random
from tensorflow import set_random_seed

RANDOM_SEED = 2018
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
set_random_seed(RANDOM_SEED)

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load Files

Load Training data

In [3]:
consumption_train = pd.read_csv('consumption_train.csv', index_col=0, parse_dates=['timestamp'])
consumption_train.head()

,series_id,timestamp,consumption,temperature
0,103088,2014-12-24 00:00:00,101842.233424,NaN
1,103088,2014-12-24 01:00:00,105878.048906,NaN
2,103088,2014-12-24 02:00:00,91619.105008,NaN
3,103088,2014-12-24 03:00:00,94473.706203,NaN
4,103088,2014-12-24 04:00:00,96976.755526,NaN


In [4]:
consumption_train.describe()

,series_id,consumption,temperature
count,509376.000000,5.093760e+05,280687.000000
mean,101851.234828,1.076238e+05,15.188679
std,1020.487674,1.626611e+05,8.987142
min,100003.000000,0.000000e+00,-13.466667
25%,100998.000000,1.542092e+04,8.451190
50%,101884.500000,4.986205e+04,15.163333
75%,102697.000000,1.351661e+05,21.800000
max,103634.000000,2.085109e+06,44.350000


In [5]:
consumption_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509376 entries, 0 to 509375
Data columns (total 4 columns):
series_id      509376 non-null int64
timestamp      509376 non-null datetime64[ns]
consumption    509376 non-null float64
temperature    280687 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 19.4 MB


Load Testing data

In [6]:
cold_start_test = pd.read_csv('cold_start_test.csv', index_col=0, parse_dates=['timestamp'])
cold_start_test.head()

,series_id,timestamp,consumption,temperature
0,102781,2013-02-27 00:00:00,15295.740389,17.00
1,102781,2013-02-27 01:00:00,15163.209562,18.25
2,102781,2013-02-27 02:00:00,15022.264079,18.00
3,102781,2013-02-27 03:00:00,15370.420458,17.00
4,102781,2013-02-27 04:00:00,15303.103213,16.90


In [7]:
cold_start_test.describe()

,series_id,consumption,temperature
count,111984.000000,1.119840e+05,67068.000000
mean,101881.195456,1.585893e+05,15.981019
std,1095.489531,5.282630e+05,7.796996
min,100004.000000,0.000000e+00,-12.212500
25%,100876.000000,1.417107e+04,10.750000
50%,101939.500000,3.869147e+04,16.000000
75%,102920.000000,1.067592e+05,21.600000
max,103629.000000,5.366167e+06,42.000000


In [8]:
cold_start_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111984 entries, 0 to 111983
Data columns (total 4 columns):
series_id      111984 non-null int64
timestamp      111984 non-null datetime64[ns]
consumption    111984 non-null float64
temperature    67068 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 4.3 MB


Loading Submission data

In [9]:
submission_format = pd.read_csv('submission_format.csv', index_col='pred_id', parse_dates=['timestamp'])
submission_format.head()

,series_id,timestamp,temperature,consumption,prediction_window
pred_id,,,,,
0,102781,2013-03-03,19.931250,0.0,daily
1,102781,2013-03-04,20.034375,0.0,daily
2,102781,2013-03-05,19.189583,0.0,daily
3,102781,2013-03-06,18.397917,0.0,daily
4,102781,2013-03-07,20.762500,0.0,daily


In [10]:
submission_format.describe()

,series_id,temperature,consumption
count,7529.000000,4579.000000,7529.0
mean,101878.964670,16.720556,0.0
std,1070.515261,7.613953,0.0
min,100004.000000,-9.000000,0.0
25%,100882.000000,11.512500,0.0
50%,101947.000000,16.645833,0.0
75%,102890.000000,22.000000,0.0
max,103629.000000,38.727273,0.0


In [11]:
submission_format.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7529 entries, 0 to 7528
Data columns (total 5 columns):
series_id            7529 non-null int64
timestamp            7529 non-null datetime64[ns]
temperature          4579 non-null float64
consumption          7529 non-null float64
prediction_window    7529 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 352.9+ KB


## Training Data

In [12]:
def describe_training_data(train_df):
    num_training_series = train_df.series_id.nunique()
    num_training_days = num_training_series * 28
    num_training_hours = num_training_days * 24
    assert num_training_hours == train_df.shape[0]
    
    desc = f'There are {num_training_series} training ' \
           f'series totaling {num_training_days} days ' \
           f'({num_training_hours} hours) of consumption data.'
    
    print(desc)
    
describe_training_data(consumption_train)

There are 758 training series totaling 21224 days (509376 hours) of consumption data.


#### choose subset of series for training

In [13]:
FRAC_TO_USE = 0.05

rng = np.random.RandomState(seed=RANDOM_SEED)
series_ids = consumption_train.series_id.unique()
training_and_validation_ids = np.random.choice(series_ids, size=int(len(series_ids)*FRAC_TO_USE), replace=False)

# reduce training data to series subset
training_and_validation_data = consumption_train[consumption_train.series_id.isin(training_and_validation_ids)]

# describe the reduced set
describe_training_data(training_and_validation_data)

There are 37 training series totaling 1036 days (24864 hours) of consumption data.


In [14]:
training_and_validation_data.head()

,series_id,timestamp,consumption,temperature
21504,103432,2015-08-26 00:00:00,138194.079984,NaN
21505,103432,2015-08-26 01:00:00,123089.088496,NaN
21506,103432,2015-08-26 02:00:00,123193.557517,NaN
21507,103432,2015-08-26 03:00:00,129756.670174,NaN
21508,103432,2015-08-26 04:00:00,127040.475610,NaN


#### split into training and validation

In [15]:
TRAIN_VALID_SPLIT = 0.2

-------------------------------------------- 1 validation set -------------------------------------------------

https://stackoverflow.com/questions/50491630/randomly-split-a-numpy-array

In [16]:
validation_ids = np.random.choice(training_and_validation_ids, int(len(training_and_validation_ids)*TRAIN_VALID_SPLIT), replace=False)
training_ids = list(set(training_and_validation_ids) - set(validation_ids))

training_data = training_and_validation_data[training_and_validation_data.series_id.isin(training_ids)]
validation_data = training_and_validation_data[training_and_validation_data.series_id.isin(validation_ids)]

In [17]:
describe_training_data(training_data)
describe_training_data(validation_data)

There are 30 training series totaling 840 days (20160 hours) of consumption data.
There are 7 training series totaling 196 days (4704 hours) of consumption data.


In [18]:
#TESTING
print(len(training_data.groupby('series_id').count()))
print(len(validation_data.groupby('series_id').count()))

30
7


--------------------------------------------------fin---------------------------------------------------------

#### KFold cross validation

In [19]:
training_and_validation_ids

array([103595, 103370, 102101, 101558, 100685, 102687, 103437, 100450,
       100807, 100142, 101591, 102345, 101075, 100399, 101967, 101926,
       102993, 102640, 100244, 100665, 100139, 103432, 101763, 103298,
       101660, 102162, 103159, 100040, 100425, 102478, 101523, 101203,
       100801, 102997, 101183, 103105, 102892], dtype=int64)

In [20]:
k = int(1/TRAIN_VALID_SPLIT)
k

5

In [21]:
cv_ids = []
from sklearn.model_selection import KFold
kf = KFold(n_splits=k, random_state=RANDOM_SEED)
for train_ids, valid_ids in kf.split(training_and_validation_ids):
    cv_ids.append((training_and_validation_ids[train_ids],training_and_validation_ids[valid_ids]))

## Preprocessing Data

In [22]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
def create_lagged_features(df, lag=1):
    if not type(df) == pd.DataFrame:
        df = pd.DataFrame(df, columns=['consumption'])
    
    def _rename_lag(ser, j):
        ser.name = ser.name + f'_{j}'
        return ser
    
    old_columns = df.columns
        
    # add a column lagged by `i` steps
    for i in range(1, lag + 1):
        df = df.join(df.consumption.shift(i).pipe(_rename_lag, i))
    
    added_features = list( set(df.columns) - set(old_columns))
    
    return df, added_features

In [24]:
def prepare_data(df, feature_generator):

    # scale training data
    consumption_scaler = MinMaxScaler(feature_range=(-1, 1))
    df['consumption'] = consumption_scaler.fit_transform(df['consumption'].values.reshape(-1, 1))

    # convert consumption series to lagged features
    df_extended, added_features = feature_generator(df)
    #TESTING
    #print(df_extended.head())
    #print(added_features)
    
    #handle missing values
    df_extended = df_extended.loc[ df_extended[added_features].dropna().index ]
    #print(df_extended.info()) #TESTING

    # X, y format taking the first column (original time series) to be the y
    X = df_extended[added_features].values
    y = df_extended.consumption
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    return X, y, consumption_scaler

In [25]:
#TESTING
X, y, consumption_scaler = prepare_data(training_data[training_data.series_id == training_data.iloc[0]['series_id']], lambda x: create_lagged_features(x,lag=24))

## Training 

In [26]:
# progress bar
from tqdm import tqdm

https://github.com/tqdm/tqdm/issues/285

In [27]:
def train_model(model, training_data, epochs=3, batch_size=1, desc="Learning Consumption Trends", lag=24):
    for i in tqdm(range(num_passes_through_data), 
                  total=num_passes_through_data, 
                  desc=desc):

        # reset the LSTM state for training on each series
        for ser_id, ser_data in training_data.groupby('series_id'):

            # prepare the data
            X, y, consumption_scaler = prepare_data(ser_data, lambda x: create_lagged_features(x,lag=lag))

            # fit the model: note that we don't shuffle batches (it would ruin the sequence)
            # and that we reset states only after an entire X has been fit, instead of after
            # each (size 1) batch, as is the case when stateful=False

            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
            model.reset_states()

## Forecasting

### Training data to cold start data

In [28]:
def train_series_to_cold_start(series_df, cold_start_days, prediction_window, window_to_num_preds=None, no_timestamp=['daily','weekly']):
    if window_to_num_preds is None:
        window_to_num_preds = {'hourly':np.arange(24),
                              'daily':np.repeat(np.arange(7),24),
                              'weekly':np.repeat(np.arange(2),24*7)}
    lengths={}
    for k,v in window_to_num_preds.items():
        lengths[k] = len(v) 
    
    cold_start_df = series_df.iloc[:cold_start_days*24]
    
    prediction_df = series_df.iloc[cold_start_days*24:]
    prediction_df['groups'] = pd.Series(window_to_num_preds[prediction_window], index=prediction_df.index[:lengths[prediction_window]])
    prediction_df = prediction_df.groupby('groups').agg({'series_id':'first','timestamp':'last','temperature':'mean','consumption':'sum'})
    
    if prediction_window in no_timestamp:
        prediction_df['timestamp'] = prediction_df['timestamp'].apply(pd.Timestamp.floor, args=('D'))
    
    prediction_df['prediction_window'] = prediction_window
    
    return cold_start_df,prediction_df

In [29]:
def get_cold_start_and_prediction_data(validation_data, prediction_window, cold_start_days=14):
    shuffled_ids = validation_data['series_id'].unique()
    np.random.shuffle(shuffled_ids)
    cold_start_data = pd.DataFrame()
    prediction_data = pd.DataFrame()
    for i in shuffled_ids:
        cold_start_df,prediction_df = train_series_to_cold_start(validation_data[validation_data.series_id==i],cold_start_days,prediction_window)
        cold_start_data = cold_start_data.append(cold_start_df)
        prediction_data = prediction_data.append(prediction_df)

    prediction_data.index = np.arange(len(prediction_data))
    
    return cold_start_data, prediction_data

## Error Metric 

In [30]:
def calc_nmae(y_df,preds_df): 
    w = {'hourly': 24/24,
        'daily':24/7,
        'weekly':24/2}
    
    preds_df['w'] = preds_df.prediction_window.map(w)
    mean_series = y_df.groupby('series_id').consumption.mean().rename('m')
    preds_df = preds_df.join(mean_series,on='series_id')
    preds_df['c'] = preds_df.w/preds_df.m
    
    nmae = (np.abs(y_df.consumption-preds_df.consumption)*preds_df.c).sum()/len(preds_df)
    return nmae

# Base Model

In [31]:
# modeling
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

Using TensorFlow backend.


In [59]:
# lag of 24 to simulate smallest cold start window. Our series
# will be converted to a num_timesteps x lag size matrix

def get_base_model(num_neurons=24,lag=24):
    batch_size = 1  # this forces the lstm to step through each time-step one at a time
    batch_input_shape=(batch_size, None, lag)

    # instantiate a sequential model
    model = Sequential()

    # add LSTM layer - stateful MUST be true here in 
    # order to learn the patterns within a series
    model.add(LSTM(units=num_neurons, 
                  batch_input_shape=batch_input_shape, 
                  stateful=True))

    # followed by a dense layer with a single output for regression
    model.add(Dense(1))

    # compile
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model

In [63]:
def generate_hourly_forecast(num_pred_hours, consumption, model, scaler, lag):
    """ Uses last hour's prediction to generate next for num_pred_hours, 
        initialized by most recent cold start prediction. Inverts scale of 
        predictions before return.
    """
    # allocate prediction frame
    preds_scaled = np.zeros(num_pred_hours)
    
    # initial X is last lag values from the cold start
    X = scaler.transform(consumption.values.reshape(-1, 1))[-lag:]
    
    # forecast
    for i in range(num_pred_hours):
        # predict scaled value for next time step
        yhat = model.predict(X.reshape(1, 1, lag), batch_size=1)[0][0]
        preds_scaled[i] = yhat
        
        # update X to be latest data plus prediction
        X = pd.Series(X.ravel()).shift(-1).fillna(yhat).values

    # revert scale back to original range
    hourly_preds = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()
    return hourly_preds

In [34]:
def get_base_model_predictions(model, cold_start_data, my_predictions, desc="Forecasting from Cold Start Data",lag=24):
    %%time
    pred_window_to_num_preds = {'hourly': 24, 'daily': 7, 'weekly': 2}
    pred_window_to_num_pred_hours = {'hourly': 24, 'daily': 7 * 24, 'weekly': 2 * 7 * 24}

    num_test_series = my_predictions.series_id.nunique()

    model.reset_states()

    for ser_id, pred_df in tqdm(my_predictions.groupby('series_id'), 
                                total=num_test_series, 
                                desc=desc):

        # get info about this series' prediction window
        pred_window = pred_df.prediction_window.unique()[0]
        num_preds = pred_window_to_num_preds[pred_window]
        num_pred_hours = pred_window_to_num_pred_hours[pred_window]

        # prepare cold start data
        series_data = cold_start_data[cold_start_data.series_id == ser_id]
        cold_X, cold_y, consumption_scaler = prepare_data(series_data, lambda x: create_lagged_features(x,lag=24))

        # fine tune our lstm model to this site using cold start data    
        model.fit(cold_X, cold_y, epochs=1, batch_size=1, verbose=0, shuffle=False)

        # make hourly forecasts for duration of pred window
        preds = generate_hourly_forecast(num_pred_hours, series_data.consumption, model, consumption_scaler, lag)

        # reduce by taking sum over each sub window in pred window
        reduced_preds = [pred.sum() for pred in np.split(preds, num_preds)]

        # store result in submission DataFrame
        ser_id_mask = my_predictions.series_id == ser_id
        my_predictions.loc[ser_id_mask, 'consumption'] = reduced_preds
        
    return my_predictions

In [35]:
batch_size=1
lag=24

In [36]:
%%time
num_training_series = training_data.series_id.nunique()
num_passes_through_data = 3
train_cmaes = []
fold_cmaes = []

for fold_num, indices in enumerate(cv_ids):
    train_ids,valid_ids = indices
    training_data = training_and_validation_data[training_and_validation_data.series_id.isin(train_ids)]
    validation_data = training_and_validation_data[training_and_validation_data.series_id.isin(valid_ids)]
    
    model = get_base_model()
    
    #train model
    train_model(model,training_data, desc=f"Learning Consumption Trends - Fold {fold_num}")
    #calculate training error
    train_cold_start_data, train_prediction_data = get_cold_start_and_prediction_data(training_data,'weekly')
    my_train_predictions = train_prediction_data.copy()    
    my_train_predictions = get_base_model_predictions(model, train_cold_start_data, my_train_predictions, desc=f"Forecasting from Training Cold Start Data - Fold {fold_num}")    
    train_cmaes.append(calc_nmae(train_prediction_data,my_train_predictions))
    
    #forecast from validation set and calculate validation error
    cold_start_data, prediction_data = get_cold_start_and_prediction_data(validation_data,'weekly')    
    my_predictions = prediction_data.copy() 
    my_predictions = get_base_model_predictions(model, cold_start_data, my_predictions, desc=f"Forecasting from Cold Start Data - Fold {fold_num}")
    fold_cmaes.append(calc_nmae(prediction_data,my_predictions))

Learning Consumption Trends - Fold 0: 100%|██████████████████████████████████████████████| 3/3 [02:25<00:00, 50.08s/it]


Wall time: 0 ns


Forecasting from Training Cold Start Data - Fold 0: 100%|██████████████████████████████| 29/29 [00:39<00:00,  1.28s/it]


Wall time: 0 ns


Learning Consumption Trends - Fold 1: 100%|██████████████████████████████████████████████| 3/3 [02:09<00:00, 43.95s/it]


...

Wall time: 0 ns


Forecasting from Training Cold Start Data - Fold 4: 100%|██████████████████████████████| 30/30 [00:54<00:00,  1.65s/it]


Wall time: 0 ns


Forecasting from Cold Start Data - Fold 4: 100%|█████████████████████████████████████████| 7/7 [00:10<00:00,  1.42s/it]


Wall time: 18min 31s


In [60]:
print(sum(train_cmaes)/len(train_cmaes))

3.97037322399


In [61]:
print(sum(fold_cmaes)/len(fold_cmaes))

4.20712859748


In [ ]:
%%time
num_training_series = training_data.series_id.nunique()
num_passes_through_data = 3
train_cmaes_1day = []
fold_cmaes_1day = []

for fold_num, indices in enumerate(cv_ids):
    train_ids,valid_ids = indices
    training_data = training_and_validation_data[training_and_validation_data.series_id.isin(train_ids)]
    validation_data = training_and_validation_data[training_and_validation_data.series_id.isin(valid_ids)]
    
    model = get_base_model()
    
    #train model
    train_model(model,training_data, desc=f"Learning Consumption Trends - Fold {fold_num}")
    #calculate training error
    train_cold_start_data, train_prediction_data = get_cold_start_and_prediction_data(training_data,'daily')
    my_train_predictions = train_prediction_data.copy()    
    my_train_predictions = get_base_model_predictions(model, train_cold_start_data, my_train_predictions, desc=f"Forecasting from Training Cold Start Data - Fold {fold_num}")    
    train_cmaes_1day.append(calc_nmae(train_prediction_data,my_train_predictions))
    
    #forecast from validation set and calculate validation error
    cold_start_data, prediction_data = get_cold_start_and_prediction_data(validation_data,'daily')    
    my_predictions = prediction_data.copy() 
    my_predictions = get_base_model_predictions(model, cold_start_data, my_predictions, desc=f"Forecasting from Cold Start Data - Fold {fold_num}")
    fold_cmaes_1day.append(calc_nmae(prediction_data,my_predictions))

In [ ]:
print(sum(train_cmaes_1day)/len(train_cmaes_1day))

In [ ]:
print(sum(fold_cmaes_1day)/len(fold_cmaes_1day))

In [62]:
%%time
num_training_series = training_data.series_id.nunique()
num_passes_through_data = 3


for fold_num, indices in enumerate(cv_ids):
    train_ids,valid_ids = indices
    training_data = training_and_validation_data[training_and_validation_data.series_id.isin(train_ids)]
    validation_data = training_and_validation_data[training_and_validation_data.series_id.isin(valid_ids)]
    
    model = get_base_model()
    
    #train model
    train_model(model,training_data, desc=f"Learning Consumption Trends - Fold {fold_num}")
    #calculate training error
    train_cold_start_data, train_prediction_data = get_cold_start_and_prediction_data(training_data,'weekly')
    my_train_predictions = train_prediction_data.copy()    
    my_train_predictions = get_base_model_predictions(model, train_cold_start_data, my_train_predictions, desc=f"Forecasting from Training Cold Start Data - Fold {fold_num}")    

    
    #forecast from validation set and calculate validation error
    cold_start_data, prediction_data = get_cold_start_and_prediction_data(validation_data,'weekly')    
    my_predictions = prediction_data.copy() 
    my_predictions = get_base_model_predictions(model, cold_start_data, my_predictions, desc=f"Forecasting from Cold Start Data - Fold {fold_num}")

    break
    

Learning Consumption Trends - Fold 0: 100%|██████████████████████████████████████████████| 3/3 [02:18<00:00, 47.22s/it]


Wall time: 0 ns


Forecasting from Training Cold Start Data - Fold 0:   0%|                                       | 0/29 [00:00<?, ?it/s]

[[-1.19988308]
 [-1.1998831 ]
 [-1.19988306]
 [-1.19988307]
 [-1.19988305]
 [-1.19988302]
 [-1.19988302]
 [-1.19988312]
 [-1.19988305]
 [-1.19988303]
 [-1.19988305]
 [-1.199883  ]
 [-1.19988302]
 [-1.19988303]
 [-1.199883  ]
 [-1.19988298]
 [-1.19988309]
 [-1.19988308]
 [-1.19988299]
 [-1.19988301]
 [-1.19988306]
 [-1.19988309]
 [-1.19988306]
 [-1.19988308]]


Forecasting from Training Cold Start Data - Fold 0:   3%|█                              | 1/29 [00:01<00:48,  1.72s/it]

...

[[-1.6625403 ]
 [-1.66253996]
 [-1.66253984]
 [-1.66254018]
 [-1.66254046]
 [-1.66254051]
 [-1.66252563]
 [-1.66251873]
 [-1.66251606]
 [-1.66251367]
 [-1.66251913]
 [-1.66252236]
 [-1.66252276]
 [-1.66252412]
 [-1.66252392]
 [-1.66252279]
 [-1.6625223 ]
 [-1.66252015]
 [-1.6625192 ]
 [-1.66251865]
 [-1.66252487]
 [-1.66253747]
 [-1.66253677]
 [-1.66253746]]


Forecasting from Cold Start Data - Fold 0:  88%|███████████████████████████████████▉     | 7/8 [00:10<00:01,  1.47s/it]

[[-1.11793811]
 [-1.11794403]
 [-1.11794382]
 [-1.11794458]
 [-1.11794428]
 [-1.11794445]
 [-1.11794441]
 [-1.11794437]
 [-1.11794093]
 [-1.1179191 ]
 [-1.11792074]
 [-1.1179217 ]
 [-1.11792086]
 [-1.11792145]
 [-1.11791994]
 [-1.11791851]
 [-1.11792015]
 [-1.11788431]
 [-1.11789497]
 [-1.11785232]
 [-1.1178449 ]
 [-1.11792124]
 [-1.11793417]
 [-1.11793396]]


Forecasting from Cold Start Data - Fold 0: 100%|█████████████████████████████████████████| 8/8 [00:12<00:00,  1.43s/it]


Wall time: 3min 30s


In [64]:
calc_nmae(train_prediction_data,my_train_predictions)

4.6096432797222082

In [65]:
calc_nmae(prediction_data,my_predictions)

3.4215315834649154

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (1, 24)                   4704      
_________________________________________________________________
dense_5 (Dense)              (1, 1)                    25        
Total params: 4,729
Trainable params: 4,729
Non-trainable params: 0
_________________________________________________________________


In [40]:
cv_ids[0][0]

array([100807, 100142, 101591, 102345, 101075, 100399, 101967, 101926,
       102993, 102640, 100244, 100665, 100139, 103432, 101763, 103298,
       101660, 102162, 103159, 100040, 100425, 102478, 101523, 101203,
       100801, 102997, 101183, 103105, 102892], dtype=int64)

In [41]:
 temp_data = training_and_validation_data[training_and_validation_data.series_id.isin(cv_ids[0][0])]

In [43]:
temp_cold_start_data, temp_prediction_data = get_cold_start_and_prediction_data(temp_data,'weekly')

In [44]:
my_test_predictions = test_prediction_data.copy()

In [47]:
 my_test_predictions = get_base_model_predictions(model, test_cold_start_data, my_test_predictions, desc=f"Forecasting from Training Cold Start Data - Fold {fold_num}")    

Wall time: 0 ns


Forecasting from Training Cold Start Data - Fold 4:   0%|                                       | 0/30 [00:00<?, ?it/s]

[[-1.19988308]
 [-1.1998831 ]
 [-1.19988306]
 [-1.19988307]
 [-1.19988305]
 [-1.19988302]
 [-1.19988302]
 [-1.19988312]
 [-1.19988305]
 [-1.19988303]
 [-1.19988305]
 [-1.199883  ]
 [-1.19988302]
 [-1.19988303]
 [-1.199883  ]
 [-1.19988298]
 [-1.19988309]
 [-1.19988308]
 [-1.19988299]
 [-1.19988301]
 [-1.19988306]
 [-1.19988309]
 [-1.19988306]
 [-1.19988308]]


Forecasting from Training Cold Start Data - Fold 4:   3%|█                              | 1/30 [00:01<00:39,  1.35s/it]

[[-2.35715591]
 [-2.357157  ]
 [-2.357157  ]
 [-2.357157  ]
 [-2.357157  ]
 [-2.35715591]
 [-2.35715265]
 [-2.35714612]
 [-2.35713851]
 [-2.35713415]
 [-2.35713524]
 [-2.35713307]
 [-2.35713415]
 [-2.35713415]
 [-2.35713198]
 [-2.35713415]
 [-2.35713415]
 [-2.35713415]
 [-2.35713742]
 [-2.35714177]
 [-2.35714286]
 [-2.35714286]
 [-2.35714938]
 [-2.357157  ]]


...

Forecasting from Training Cold Start Data - Fold 4:  93%|████████████████████████████  | 28/30 [00:42<00:03,  1.69s/it]

[[-1.6625403 ]
 [-1.66253996]
 [-1.66253984]
 [-1.66254018]
 [-1.66254046]
 [-1.66254051]
 [-1.66252563]
 [-1.66251873]
 [-1.66251606]
 [-1.66251367]
 [-1.66251913]
 [-1.66252236]
 [-1.66252276]
 [-1.66252412]
 [-1.66252392]
 [-1.66252279]
 [-1.6625223 ]
 [-1.66252015]
 [-1.6625192 ]
 [-1.66251865]
 [-1.66252487]
 [-1.66253747]
 [-1.66253677]
 [-1.66253746]]


Forecasting from Training Cold Start Data - Fold 4:  97%|█████████████████████████████ | 29/30 [00:43<00:01,  1.68s/it]

[[-1.11793811]
 [-1.11794403]
 [-1.11794382]
 [-1.11794458]
 [-1.11794428]
 [-1.11794445]
 [-1.11794441]
 [-1.11794437]
 [-1.11794093]
 [-1.1179191 ]
 [-1.11792074]
 [-1.1179217 ]
 [-1.11792086]
 [-1.11792145]
 [-1.11791994]
 [-1.11791851]
 [-1.11792015]
 [-1.11788431]
 [-1.11789497]
 [-1.11785232]
 [-1.1178449 ]
 [-1.11792124]
 [-1.11793417]
 [-1.11793396]]


Forecasting from Training Cold Start Data - Fold 4: 100%|██████████████████████████████| 30/30 [00:45<00:00,  1.59s/it]


In [50]:
a = np.array([np.random.random() for i in range(24)])
av = np.vstack([a,a+0.5])
print(av,av.shape)

[[ 0.69526687  0.45343108  0.15580382  0.38386193  0.7107313   0.38781565
   0.84662046  0.81854201  0.77314714  0.93830505  0.46006457  0.08628216
   0.043596    0.68453984  0.41614401  0.17837524  0.38305887  0.26613085
   0.29869747  0.90025491  0.87572807  0.49391544  0.66586234  0.6130743 ]
 [ 1.19526687  0.95343108  0.65580382  0.88386193  1.2107313   0.88781565
   1.34662046  1.31854201  1.27314714  1.43830505  0.96006457  0.58628216
   0.543596    1.18453984  0.91614401  0.67837524  0.88305887  0.76613085
   0.79869747  1.40025491  1.37572807  0.99391544  1.16586234  1.1130743 ]] (2, 24)


In [57]:
av=np.array([av])
print(av.shape)

(1, 2, 24)


In [66]:
model.predict(av, batch_size=1)

array([[ 0.3579101]], dtype=float32)

# Model 2

https://stackoverflow.com/questions/51720236/multi-feature-sequence-padding-and-masking-in-rnn-using-keras

https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras